# 🍍🍅 Fruit & Vegetable Classifier - Web UI

This notebook provides an interactive web interface to classify fruits and vegetables using a pre-trained model.

**No training required** - just upload the `FV.h5` model file and start classifying!

## 1. Install Dependencies

In [ ]:
# Install required packages
!pip install -q tensorflow pillow numpy gradio

## 2. Import Libraries

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image
import gradio as gr

print(f"TensorFlow version: {tf.__version__}")

## 3. Upload Model File

Upload your `FV.h5` model file here.

In [ ]:
from google.colab import files

print("Upload your FV.h5 model file:")
uploaded = files.upload()

# Get the uploaded filename
model_path = list(uploaded.keys())[0]
print(f"\n✅ Model uploaded: {model_path}")

## 4. Load Model

In [ ]:
# Load the pre-trained model
model = load_model(model_path)
print("✅ Model loaded successfully!")

# Model parameters
IMG_SIZE = 224

# Class labels
labels = {
    0: 'apple', 1: 'banana', 2: 'beetroot', 3: 'bell pepper', 4: 'cabbage', 
    5: 'capsicum', 6: 'carrot', 7: 'cauliflower', 8: 'chilli pepper', 9: 'corn', 
    10: 'cucumber', 11: 'eggplant', 12: 'garlic', 13: 'ginger', 14: 'grapes', 
    15: 'jalepeno', 16: 'kiwi', 17: 'lemon', 18: 'lettuce', 19: 'mango', 
    20: 'onion', 21: 'orange', 22: 'paprika', 23: 'pear', 24: 'peas', 
    25: 'pineapple', 26: 'pomegranate', 27: 'potato', 28: 'raddish', 
    29: 'soy beans', 30: 'spinach', 31: 'sweetcorn', 32: 'sweetpotato', 
    33: 'tomato', 34: 'turnip', 35: 'watermelon'
}

print(f"\nModel supports {len(labels)} classes")

## 5. Launch Interactive Web UI

In [ ]:
def predict_fruit_vegetable(image):
    """
    Predict fruit/vegetable from uploaded image
    
    Args:
        image: PIL Image or numpy array
    
    Returns:
        Formatted prediction results
    """
    # Ensure image is PIL Image
    if not isinstance(image, Image.Image):
        image = Image.fromarray(image)
    
    # Resize and preprocess
    img = image.resize((IMG_SIZE, IMG_SIZE))
    img_array = img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    predicted_class = np.argmax(predictions[0])
    confidence = predictions[0][predicted_class]
    
    # Get class name
    class_name = labels[predicted_class].capitalize()
    
    # Determine category
    fruits = ['Apple', 'Banana', 'Bell Pepper', 'Chilli Pepper', 'Grapes', 'Jalepeno', 
              'Kiwi', 'Lemon', 'Mango', 'Orange', 'Paprika', 'Pear', 'Pineapple', 
              'Pomegranate', 'Watermelon']
    
    category = "🍎 Fruit" if class_name in fruits else "🥬 Vegetable"
    
    # Get top 3 predictions
    top_3_idx = np.argsort(predictions[0])[-3:][::-1]
    top_3_results = {
        labels[idx].capitalize(): float(predictions[0][idx]) 
        for idx in top_3_idx
    }
    
    # Format result
    result = f"""
### Prediction: {class_name}
**Category:** {category}  
**Confidence:** {confidence*100:.2f}%

#### Top 3 Predictions:
"""
    for name, conf in top_3_results.items():
        result += f"\n- **{name}**: {conf*100:.2f}%"
    
    return result

# Create Gradio interface
demo = gr.Interface(
    fn=predict_fruit_vegetable,
    inputs=gr.Image(type="pil", label="Upload Fruit or Vegetable Image"),
    outputs=gr.Markdown(label="Prediction Results"),
    title="🍍🍅 Fruit & Vegetable Classifier",
    description="Upload an image of a fruit or vegetable to classify it! Supports 36 different classes.",
    theme="soft",
    allow_flagging="never"
)

# Launch the interface
print("🚀 Launching interactive web UI...")
print("📱 The interface will open below and provide a public URL you can share!")
demo.launch(share=True, debug=False)